In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import csv
import os
import pickle
from pathlib import Path
from tqdm import tqdm
from PIL import ImageOps, Image

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

# Push-up

## Get images path
Only pushup-up class

In [3]:
image_dirs = ['/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up']
accepted_exts = ('.jpg', '.jpeg','.png')
all_file_paths = []
for image_dir in image_dirs:
    for path, subdir, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(accepted_exts):
                full_path = os.path.join(path, file)
                print(full_path)
                all_file_paths.append(full_path)

/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/64_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/27_1_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/67_1_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/86_2_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/42_6_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/28_4_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/11_2_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/68_1_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/69_3_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/7_5_1.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/27_2_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/24_10_2.jpg
/home/u6188041/jom/Dataset/EzFit_dataset/images/pushup/pushup-up/101_1_2.jpg
/home/u6188

In [4]:
len(all_file_paths)

438

## Extract keypoints from path

In [ ]:
SEQ_LEN = 8
data = []
error_files = []
for image_path in tqdm(all_file_paths):
    image = cv2.imread(image_path)
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose2:
            results = pose.process(image)

            try:
                pose_coor = results.pose_landmarks.landmark
                # no need z here
                body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
                data.append(stacked_kps)

            except:
                error_files.append(image_path)
                
            # augment flip horizontal
            flipped_image = np.flip(image, axis=1)   
            aug_results = pose2.process(flipped_image)
            
            try:
                pose_coor = aug_results.pose_landmarks.landmark
                # no need z here
                body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                stacked_kps = np.tile(body_kps, (SEQ_LEN, 1))
                data.append(stacked_kps)

            except:
                error_files.append('aug_'+image_path)

  0%|                                                                                           | 0/438 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
  0%|▏                                                                                  | 1/438 [00:00<01:26,  5.06it/s]INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 318 node(s) with delega

In [ ]:
error_files

In [ ]:
data = np.array(data)
data.shape

In [ ]:
1500/474

In [ ]:
# stack data more
STACK_MORE = 3
stacked_data = np.tile(data, (STACK_MORE, 1))
stacked_data.shape

In [ ]:
stacked_data = stacked_data.reshape((-1, 8, 99))
stacked_data

In [ ]:
stacked_data.shape

In [ ]:
np.min(stacked_data), np.mean(stacked_data),np.max(stacked_data)

In [ ]:
np.random.seed(42)
noise = np.random.normal(scale=0.01, size=stacked_data.shape[:3])
noise.shape

In [ ]:
noise.min(), noise.mean(), noise.max()

In [ ]:
noisy_stacked_data = stacked_data + noise

In [ ]:
noisy_stacked_data.min(), noisy_stacked_data.mean(),noisy_stacked_data.max()

In [17]:
np.save('saved_data/half-burpee_idle_class_len_8', noisy_stacked_data)

# Save to .npy files

In [18]:
idle_data = np.load('saved_data/half-burpee_idle_class_len_8.npy')
idle_data.shape

(1422, 8, 99)

In [19]:
idle_data[0,:].shape

(8, 99)

In [20]:
save_dir = Path('D:/EzFit_dataset/keypoints_seq/len_8/half-burpee/idle')
for idx in tqdm(range(idle_data.shape[0])):
    base_dir = save_dir / str(idx)
    if not base_dir.exists():
        base_dir.mkdir(parents=True)
    selected = idle_data[idx,:]
    np.save(base_dir / f'idle_{idx}', selected)

100%|█████████████████████████████████████████████████████████████████████████████| 1422/1422 [00:03<00:00, 440.29it/s]
